<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# FEC - Création du dataset "Bilan"

**Tags:** #fec #finance #snippet #operation #bilan

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** Ce notebook permet de créer le dataset pour visualiser la décomposition du bilan.

## Input

### Import libraries

In [1]:
import naas_data_product

✅ utils file '/home/ftp/FEC-engine/utils/datalake.ipynb' successfully loaded.
✅ utils file '/home/ftp/FEC-engine/utils/openai.ipynb' successfully loaded.


### Setup Variables

In [2]:
# Inputs
input_folder_path = "/home/ftp/FEC-engine/outputs/FEC/bdd_fin_enr"
openai_api_key = naas.secret.get(name="OPENAI_API_KEY") or "ENTER_YOUR_OPENAI_API_KEY"

# Outputs
output_folder_path = "/home/ftp/FEC-engine/outputs/FEC/dataset_bilan"
prompt_data_n = """
Voici le jeu de données des catégories du bilan d'une entreprise sur une année:
- les différentes années dans la colonne 'PERIOD'
- les types de rubriques Bilan dans la colonne "RUBRIQUE_N0", "RUBRIQUE_N1", "RUBRIQUE_N2"
- les valeurs de l'année N en colonne 'VALUE'
Les valeurs du Bilan total de l'année N est de [BILAN_N], tandis que la valeur N-1 est [BILAN_N-1].
- Peux-tu me faire une bref aperçu factuel de l'évolution des masses du bilan entre N et N-1?
"""

prompt_data_n_1 = """
Voici le jeu de données des catégories du bilan d'une entreprise sur une année:
- les différentes années dans la colonne 'PERIOD'
- les types de rubriques Bilan dans la colonne "RUBRIQUE_N0", "RUBRIQUE_N1", "RUBRIQUE_N2"
- les valeurs de l'année N en colonne 'VALUE'
Les valeurs du Bilan total de l'année N est de [BILAN_N].
- Peux-tu me faire une analyse de la répartition du bilan par rubriques?
"""

## Model

### Récupération du dernier fichier input

In [3]:
df_input = get_last_df(input_folder_path)
print("✅ Row fetched:", len(df_input))
df_input.head(1)

📁 Last file: /home/ftp/FEC-engine/outputs/FEC/bdd_fin_enr/20230622115039_FEC_bdd_fin_enr.csv
✅ Row fetched: 888


,ENTITY,PERIOD,COMPTE_NUM,RUBRIQUE_N0,RUBRIQUE_N1,RUBRIQUE_N2,RUBRIQUE_N3,VALUE,VALUE_N-1,VARV,VARP
0,000000000,201712,10130000,PASSIF,PASSIF_NON_COURANT,CAPITAUX_PROPRES,CAPITAL SOUSCRIT APPELÉ VERSÉ,-6000.0,0.0,-6000.0,-inf


### Création du dataset "BILAN"

In [4]:
def prep_data(df_init):
    # Init
    df = df_init.copy()
    
    # Filtre RUBRIQUE_N0 = ACTIF & PASSIF
    df = df[(df["RUBRIQUE_N0"].isin(["ACTIF", "PASSIF"]))]

    # Regroupement R0/R1/R2
    to_group = ["ENTITY", "PERIOD", "RUBRIQUE_N0", "RUBRIQUE_N1", "RUBRIQUE_N2"]
    to_agg = {"VALUE": "sum"}
    df = df.groupby(to_group, as_index=False).agg(to_agg).fillna(0)

    # Mettre en valeur positive VALUE
    df["VALUE"] = df["VALUE"].abs()
    df["VALUE_D"] = (df["VALUE"] / 1000).map("{:,.1f} k€".format).str.replace(",", " ")
    return df.reset_index(drop=True)

data_prep = prep_data(df_input)
print("Row fetched:", len(data_prep))
data_prep.head(20)

Row fetched: 54


,ENTITY,PERIOD,RUBRIQUE_N0,RUBRIQUE_N1,RUBRIQUE_N2,VALUE,VALUE_D
0,000000000,201712,ACTIF,ACTIF_COURANT,AUTRES_CREANCES,25148.74,25.1 k€
1,000000000,201712,ACTIF,ACTIF_COURANT,CREANCES_CLIENTS,38959.55,39.0 k€
2,000000000,201712,ACTIF,ACTIF_COURANT,STOCKS,4030.77,4.0 k€
3,000000000,201712,ACTIF,ACTIF_NON_COURANT,IMMOBILISATIONS,79527.73,79.5 k€
4,000000000,201712,ACTIF,DISPONIBILITES,AUTRES DISPONIBILITES,0.00,0.0 k€
5,000000000,201712,ACTIF,DISPONIBILITES,BANQUES & CAISSE,129172.30,129.2 k€
6,000000000,201712,PASSIF,PASSIF_COURANT,AUTRES_DETTES,56471.60,56.5 k€
7,000000000,201712,PASSIF,PASSIF_COURANT,DETTES_FOURNISSEURS,1596.78,1.6 k€
8,000000000,201712,PASSIF,PASSIF_NON_COURANT,CAPITAUX_PROPRES,218770.71,218.8 k€
9,000000000,201812,ACTIF,ACTIF_COURANT,AUTRES_CREANCES,26039.50,26.0 k€


### Création des analyses N et N-1 par période

In [5]:
def create_data_bilan(df_init):
    # Init
    df = df_init.copy()
    df_output = pd.DataFrame()
    
    # Get periods by entity
    entities = df[["ENTITY"]].drop_duplicates(ignore_index=True)
    for row in entities.itertuples():
        entity = row.ENTITY
        tmp_df = df.copy()
        tmp_df = tmp_df[(tmp_df["ENTITY"] == entity)]
        
        # Get periods
        periods = tmp_df[["PERIOD"]].drop_duplicates(ignore_index=True)
        
        for i, r in periods.iterrows():
            bilan_n = 0
            bilan_n_1 = 0
            period = periods["PERIOD"][i]
            if i < len(periods) - 1:
                period_comp = periods["PERIOD"][i+1]
            else:
                period_comp = None
            print(f"➡️ Entity: {entity}, Periode: {period_comp} -> {period}")
            bilan_n = "{:,.1f} k€".format(tmp_df[(tmp_df["PERIOD"] == period) & (tmp_df["RUBRIQUE_N0"] == "ACTIF")]["VALUE"].sum()).replace(",", " ")
            bilan_n_1 = "{:,.1f} k€".format(tmp_df[(tmp_df["PERIOD"] == period_comp) & (tmp_df["RUBRIQUE_N0"] == "ACTIF")]["VALUE"].sum()).replace(",", " ")
            print(bilan_n, bilan_n_1)
            if bilan_n_1 != "0.0 k€":
                prompt_data = prompt_data_n
            else:
                prompt_data = prompt_data_n_1
                
            tmp_df2 = tmp_df[(tmp_df["PERIOD"].isin([period, period_comp]))].reset_index(drop=True)
            print("Row fetched:", len(tmp_df2))                                                                      
            analysis = get_ia_analysis(openai_api_key, tmp_df2, prompt_data.replace("[BILAN_N]", str(bilan_n)).replace("[BILAN_N-1]", str(bilan_n_1)))
            tmp_df2["AI_ANALYSIS"] = analysis
            
            # Concat
            df_output = pd.concat([df_output, tmp_df2])
    return df_output.reset_index(drop=True)

df_bilan = create_data_bilan(data_prep)
print("Row fetched:", len(df_bilan))
df_bilan.head(24)

➡️ Entity: 000000000, Periode: 201812 -> 201712
276 839.1 k€ 352 723.8 k€
Row fetched: 18
🤖 IA: Entre N et N-1, le bilan total de l'entreprise a diminué de 75 884.7 k€, passant de 352 723.8 k€ à 276 839.1 k€. Cette baisse est principalement due à une diminution des immobilisations de 35 541.37 k€ et des disponibilités de 73 184.78 k€. En revanche, les créances clients ont augmenté de 3 616.08 k€ et les capitaux propres de 39 057.85 k€.
➡️ Entity: 000000000, Periode: None -> 201812
352 723.8 k€ 0.0 k€
Row fetched: 9
🤖 IA: Sur l'année N, le bilan total de l'entreprise est de 352 723.8 k€. La répartition du bilan par rubriques montre que les immobilisations représentent la plus grande part avec 75.9 k€, suivies des capitaux propres avec 251.6 k€. Les disponibilités en banques et caisse représentent également une part importante avec 204 k€. Les autres rubriques ont des valeurs moins significatives.
➡️ Entity: 000000001, Periode: 202212 -> 202112
276 839.1 k€ 352 723.8 k€
Row fetched: 18
🤖

,ENTITY,PERIOD,RUBRIQUE_N0,RUBRIQUE_N1,RUBRIQUE_N2,VALUE,VALUE_D,AI_ANALYSIS
0,000000000,201712,ACTIF,ACTIF_COURANT,AUTRES_CREANCES,25148.74,25.1 k€,"Entre N et N-1, le bilan total de l'entreprise..."
1,000000000,201712,ACTIF,ACTIF_COURANT,CREANCES_CLIENTS,38959.55,39.0 k€,"Entre N et N-1, le bilan total de l'entreprise..."
2,000000000,201712,ACTIF,ACTIF_COURANT,STOCKS,4030.77,4.0 k€,"Entre N et N-1, le bilan total de l'entreprise..."
3,000000000,201712,ACTIF,ACTIF_NON_COURANT,IMMOBILISATIONS,79527.73,79.5 k€,"Entre N et N-1, le bilan total de l'entreprise..."
4,000000000,201712,ACTIF,DISPONIBILITES,AUTRES DISPONIBILITES,0.00,0.0 k€,"Entre N et N-1, le bilan total de l'entreprise..."
5,000000000,201712,ACTIF,DISPONIBILITES,BANQUES & CAISSE,129172.30,129.2 k€,"Entre N et N-1, le bilan total de l'entreprise..."
6,000000000,201712,PASSIF,PASSIF_COURANT,AUTRES_DETTES,56471.60,56.5 k€,"Entre N et N-1, le bilan total de l'entreprise..."
7,000000000,201712,PASSIF,PASSIF_COURANT,DETTES_FOURNISSEURS,1596.78,1.6 k€,"Entre N et N-1, le bilan total de l'entreprise..."
8,000000000,201712,PASSIF,PASSIF_NON_COURANT,CAPITAUX_PROPRES,218770.71,218.8 k€,"Entre N et N-1, le bilan total de l'entreprise..."
9,000000000,201812,ACTIF,ACTIF_COURANT,AUTRES_CREANCES,26039.50,26.0 k€,"Entre N et N-1, le bilan total de l'entreprise..."


## Output

### Sauvegarde des fichiers en csv

In [6]:
save_df(df_bilan, output_folder_path)

✅ DataFrame saved in: /home/ftp/FEC-engine/outputs/FEC/dataset_bilan/20230626125458_FEC_dataset_bilan.csv
